In [1]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization

SOL_df = data.load_asset('SOLUSDT', sampling='6h')
SOL_df = data.subset(SOL_df, start=pd.Timestamp('2021-01-01 08:00:00'), end=pd.Timestamp('2026-01-01'))
data.report_and_print_gaps(SOL_df, delta=pd.Timedelta('6h'))

SOL_df = data.add_returns(SOL_df)
SOL_df = data.add_fear_and_greed(SOL_df, includeCategory=False)
SOL_df.dropna(inplace=True)
SOL_df = data.train_test_split(SOL_df)
print(SOL_df.columns)


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume',
       'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'Return', 'Return_Target', 'F&G',
       'SPLIT'],
      dtype='object')


In [2]:
# vs. normalize all features AND targets
DF = SOL_df.copy()

numerical_columns = categorical_columns = "all"
exclude_columns = []
DF_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'F&G', 'Return']
DF_features_simple = ['Open', 'High', 'Low', 'Close', 'Volume', 'Return']
DF_features_nosentiment = ['Open', 'High', 'Low', 'Close', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'Return']
DF, DF_columns = data.normalize_data(DF, exclude_columns=exclude_columns)

DF_columns

{'Open': MinMaxScaler(),
 'High': MinMaxScaler(),
 'Low': MinMaxScaler(),
 'Close': MinMaxScaler(),
 'Volume': MinMaxScaler(),
 'Quote asset volume': MinMaxScaler(),
 'Number of trades': MinMaxScaler(),
 'Taker buy base asset volume': MinMaxScaler(),
 'Taker buy quote asset volume': MinMaxScaler(),
 'Return': MinMaxScaler(),
 'Return_Target': MinMaxScaler(),
 'F&G': MinMaxScaler(),
 'SPLIT': 'standard'}

In [13]:
import json
import optuna
from IPython.display import display, HTML
import joblib

importlib.reload(LSTM_returns)

Log = []

def objective(trial):
    entry = {}
    # Example of hyperparameter suggestions
    hyperparams = {
        'lstm_units': trial.suggest_int('lstm_units', 50, 90),
        'num_layers': trial.suggest_int('num_layers', 3, 5),
        'epochs': 1,
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128]),
        'dropout': trial.suggest_int('dropout_rate', 0, 2) / 20,
        'learning_rate': trial.suggest_categorical('learning_rate', [0.00001, 0.0001, 0.001, 0.01, 0.1]),
        'LSTM_activation': trial.suggest_categorical('lstm_activation', ['relu', 'tanh']),
        '__+ABD_exceptLast': False,
        '__finalAct': 'sigmoid',
        'optimizer': 'adam',
        'features': trial.suggest_categorical('features', ['all', 'simple', 'nosentiment']),
        # 'loss': trial.suggest_categorical('loss', ['mean_squared_error', 'mean_absolute_error', 'huber']),
    }
    episode_lengths = trial.suggest_int('episode_length', 10, 60)
    hyperparams['episode_length'] = episode_lengths
    hyperparams['layers'] = [hyperparams['lstm_units']] * hyperparams['num_layers']
    hyperparams['loss'] = 'mean_squared_error'
    # hyperparams['batch_size'] = 128
    # hyperparams['epochs'] = 1  # Set to 1 for quick testing; increase for actual training

    
    # Placeholder for model training and evaluation
    features = {
        'all': DF_features,
        'simple': DF_features_simple,
        'nosentiment': DF_features_nosentiment
    }[hyperparams['features']]

    train_X_y = LSTM_returns.vectorize_train_data(DF, features=features, episode_length=episode_lengths)
    hyperparams['input_shape'] = train_X_y['train_X'].shape[1:]
    
    LSTM = LSTM_returns.create(hyperparams)
    results = LSTM_returns.train(LSTM, hyperparams, train_X_y, evaluateAtEnd=False)

    entry['hyperparams'] = hyperparams
    entry['results'] = results
    entry['plot-title'] = f"EL: {episode_lengths}; HP: {json.dumps(hyperparams)}"
    entry['model'] = LSTM
    entry['train_X_y'] = train_X_y
    Log.append(entry)

    return results.history['loss'][-1]  # Return the last loss value as the score


# Example Optuna study
study = optuna.create_study(direction='minimize')  # or 'maximize' depending on the objective

trial_supplement = []

for i in range(50):
    study.optimize(objective, n_trials=1)

    # best_entry = [
    #     entry for entry in Log if entry['results'].history['loss'][-1] == study.best_value
    # ][-1]

    latest_entry = Log[-1]
    Log = []

    # Print the best hyperparameters
    # Display the best hyperparameters and score in purple
    display(HTML(f"""
    <div style="color: purple; background: orange; font-weight: bold;">
        Trial {i*10+1} completed.<br>
        Best hyperparameters: {study.best_params}<br>
        Best score (MSE): {study.best_value}<br>
    </div>
    """))

    # Display the best entry plot
    plt = LSTM_returns.all_folds_plot(
        latest_entry['model'], latest_entry['train_X_y']
    )
    plt.title(latest_entry['plot-title'][:175])
    print(latest_entry['plot-title'])

    file_name = f"trials-after-1.6/stupid {i*10+1:05}.png"
    plt.savefig(file_name, dpi=150, bbox_inches='tight')
    trial_supplement.append({'file_name': file_name})
    plt.close()

[I 2025-04-28 08:42:01,881] A new study created in memory with name: no-name-0b9ff244-c7ad-49c2-8c22-a165b8fc251d
c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - loss: 0.0040 - mae: 0.0470


[I 2025-04-28 08:42:05,998] Trial 0 finished with value: 0.0029947375878691673 and parameters: {'lstm_units': 76, 'num_layers': 3, 'batch_size': 128, 'dropout_rate': 1, 'learning_rate': 0.01, 'lstm_activation': 'tanh', 'features': 'all', 'episode_length': 50}. Best is trial 0 with value: 0.0029947375878691673.


154/154 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
EL: 50; HP: {"lstm_units": 76, "num_layers": 3, "epochs": 1, "batch_size": 128, "dropout": 0.05, "learning_rate": 0.01, "LSTM_activation": "tanh", "__+ABD_exceptLast": false, "__finalAct": "sigmoid", "optimizer": "adam", "features": "all", "episode_length": 50, "layers": [76, 76, 76], "loss": "mean_squared_error", "input_shape": [50, 10]}


c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0047 - mae: 0.0525


[I 2025-04-28 08:42:12,130] Trial 1 finished with value: 0.0032679361756891012 and parameters: {'lstm_units': 56, 'num_layers': 4, 'batch_size': 128, 'dropout_rate': 0, 'learning_rate': 0.01, 'lstm_activation': 'relu', 'features': 'nosentiment', 'episode_length': 24}. Best is trial 0 with value: 0.0029947375878691673.


101/155 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

KeyboardInterrupt: 

In [15]:
# Extract trial parameters and results
trials_data = [
    {**trial.params, 'score (MSE)': trial.value, 'file': supplement.get('file_name', '')}
    for (trial, supplement) in zip(study.trials, trial_supplement)
]

# Convert to a DataFrame
trials_df = pd.DataFrame(trials_data)

# Display the DataFrame
df = trials_df.sort_values(by="score (MSE)")
df.to_csv('trials_results.csv', index=False)
df

,lstm_units,num_layers,batch_size,dropout_rate,learning_rate,lstm_activation,features,episode_length,score (MSE),file
0,76,3,128,1,0.01,tanh,all,50,0.002995,trials-after-1.6/stupid 00001.png
